# Writing a module

Calculation modules are most often subclasses of the *progressivis.table.module.TableModule* class. A lower level class exists (*progressivis.core.Module*) but it is unsuitable for direct use.

As a general rule, to define a new module you must:

1. Create a class deriving TableModule
2. Define the input slots on this class via a class attribute called `inputs`
3. Define on this class, only if necessary, additional output slots via a class attribute called `outputs` , knowing that the default output named `"table"`  is already defined in TableModule and inherited by your class.
4. Define the `run_step()` method on this class


In [1]:
from progressivis.table.module import TableModule, Table
from progressivis import SlotDescriptor

class MyModule(TableModule):
    inputs = [SlotDescriptor('first', type=Table, required=True), 
              SlotDescriptor('second', type=Table, required=True)]
    # defining the "outputs" attribute is not necessary when 
    # only one output is requested. Just use the already defined "table" output
    def run_step(self, run_number, step_size, howlong):
        """
        * run_number: identifies the current run step
        * step_size: number of rows to be processed in this step
        * howlong: assigned time for this step
        """ 
        pass # your algorithm 

## Writing a `run_step()` method 

Regardless of the algorithm implemented, we can identify eight parts in the method which will be more or less well separated:

1. Analysis of input slots
2. Special actions if the state of the slots requires it
3. Choice of the size of the data that will be consumed on the slots
4. Effective reading on slots
5. Processing itself
6. Creation of output objects (Table, PsDict, only once, before the first output write)
7. Writing output data
8. Return of the next module state and number of steps taken

**NB:** in the next examples focused lines are commented like that: # /!\

### Analyze input slots

Analysis is possible via several methods available on an input slot object.

Access to the entry slots can be done by name:

In [2]:
class MyModule(TableModule):
    inputs = [SlotDescriptor('first', type=Table, required=True), 
              SlotDescriptor('second', type=Table, required=True)]
    def run_step(self, run_number, step_size, howlong):
        slot = self.get_input_slot('first') # /!\
        # ...

We can also process slots in a loop:

In [3]:
class MyModule(TableModule):
    inputs = [SlotDescriptor('first', type=Table, required=True), 
              SlotDescriptor('second', type=Table, required=True)]
    def run_step(self, run_number, step_size, howlong):
        for slot_name, slot in self._input_slots.items(): # /!\
            if slot is None: # e.g. _params slot not provided
                continue
            # ...

Available methods:

* `slot.has_buffered()`: informs if data has been created / modified / deleted, without distinction since the last run
* `slot.created.any()`: data has been created since the last run
* `slot.updated.any()`: data has been modified since the last run
* `slot.deleted.any()`: data has been deleted since the last run
* `slot.created.length()`: number of rows created since the last run
* `slot.updated.length()`: number of rows modified since the last run
* `slot.deleted.length()`: number of rows deleted since the last run

All of these methods are idempotent.

### Special actions if the state of the slots requires it

After analysis, depending on the state of the inputs and the semantics of your module, several situations may arise:

* If one or more slots have not changed since the last execution (according to your processing needs) the execution stops and the module switches to "blocked" state (waiting for new data):

In [4]:
class MyModule(TableModule):
    inputs = [SlotDescriptor('first', type=Table, required=True), 
              SlotDescriptor('second', type=Table, required=True)]
    def run_step(self, run_number, step_size, howlong):
        slot = self.get_input_slot('first')
        if not slot.has_buffered():
            return self._return_run_step(self.state_blocked, steps_run=0) # /!\
        # ...

* Most of the time, the modification or deletion of data already processed invalidates the result already produced. When this happens, your `run_step()` method should:
     * repair the results already produced when possible
     * or reinitialize the concerned slot (and sometimes other slots involved in the operation) as well as the output and start the processing from scratch.

Repair, when possible, is closely related to the semantics of your algorithm so this is not covered here, but the reset operation is algorithm-agnostic:

In [5]:
class Max(TableModule):
    inputs = [SlotDescriptor('table', type=Table, required=True)]
    def run_step(self, run_number, step_size, howlong):
        slot = self.get_input_slot('table')
        if slot.updated.any() or slot.deleted.any():
            slot.reset()
            if self._table is not None:
                self._table.resize(0)
            slot.update(run_number)
        # ...

**NB:** sometimes resetting only the impacted slot is not sufficient:

In [6]:
class Hadamard(TableModule):
    inputs = [SlotDescriptor('x1', type=Table, required=True), 
              SlotDescriptor('x2', type=Table, required=True)]
    def run_step(self, run_number, step_size, howlong):
        x1 = self.get_input_slot('x1')
        x2 = self.get_input_slot('x2')
        if x1.updated.any() or x1.deleted.any() or x2.updated.any() or x2.deleted.any():
            x1.reset()
            x2.reset()
            if self._table is not None:
                self._table.resize(0)
            x1.update(run_number)        
            x2.update(run_number)
        # ... process element-wise x1 * x2

**NB:** Always call slot.update() after a slot.reset()

### Choice of the size of the data that will be consumed on the slots

When there is only one input slot, the data size is given by the `step_size` argument but when there are several slots, you have to choose the size of data that you are able to process in a step:

In [7]:
class Hadamard(TableModule):
    inputs = [SlotDescriptor('x1', type=Table, required=True), 
              SlotDescriptor('x2', type=Table, required=True)]
    def run_step(self, run_number, step_size, howlong):
        x1 = self.get_input_slot('x1')
        x2 = self.get_input_slot('x2')
        if x1.updated.any() or x1.deleted.any() or x2.updated.any() or x2.deleted.any():
            x1.reset()
            x2.reset()
            if self._table is not None:
                self._table.resize(0)
            x1.update(run_number)        
            x2.update(run_number)
        step_size = min(x1.created.length(), x2.created.length(), step_size) # /!\
        # ... process element-wise x1 * x2

### Effective reading on the slots

The key method in this step is the `next()` method called with the size estimated in the previous step. 
Calling `slot.created.next(step_size)` can get `n <=step_size` rows if` slot.created.length () <=step_size` but never more rows than asked:

In [8]:
class Max(TableModule):
    inputs = [SlotDescriptor('table', type=Table, required=True)]
    def run_step(self, run_number, step_size, howlong):
        slot = self.get_input_slot('table')
        if slot.updated.any() or slot.deleted.any():
            slot.reset()
            if self._table is not None:
                self._table.resize(0)
            slot.update(run_number)
        indices = slot.created.next(step_size) # /!\ 
        steps = indices_len(indices)
        if steps==0:
            return self._return_run_step(self.state_blocked, steps_run=0)
        # ...

And also:

In [9]:
class Hadamard(TableModule):
    inputs = [SlotDescriptor('x1', type=Table, required=True), 
              SlotDescriptor('x2', type=Table, required=True)]
    def run_step(self, run_number, step_size, howlong):
        x1 = self.get_input_slot('x1')
        x2 = self.get_input_slot('x2')
        if x1.updated.any() or x1.deleted.any() or x2.updated.any() or x2.deleted.any():
            x1.reset()
            x2.reset()
            if self._table is not None:
                self._table.resize(0)
            x1.update(run_number)        
            x2.update(run_number)
        step_size = min(x1.created.length(), x2.created.length(), step_size) 
        x1_indices = x1.created.next(step_size) # /!\ 
        x2_indices = x2.created.next(step_size) # /!\ 
        # ... process element-wise x1 * x2

### The treatment itself

This part is related to your algorithm. You can access the data of a slot via the `data()` method. The data associated with a slot are most of the time `Table` objects and sometimes` PsDict` objects. `Table` objects have a ** Pandas-alike ** interface (a subset, actually) while` PsDict` are Python dictionaries (`PsDict` inherits from` dict`).

As the `next()` method could return a slice (numpy-alike) when the indices are contiguous or the set of indices if they are not (actually a Roaring bitmap), two other functions are important for this step:

* `indices_len(indices)`: get the number of indices regardless of the nature of `indices` (slice or bitmap)
* `fix_loc(indices)` : get a valid Pandas-alike `loc` index regardless of the nature of `indices` (slice or bitmap)

For example:

In [10]:
class Max(TableModule):
    inputs = [SlotDescriptor('table', type=Table, required=True)]
    def run_step(self, run_number, step_size, howlong):
        slot = self.get_input_slot('table')
        if slot.updated.any() or slot.deleted.any():
            slot.reset()
            if self._table is not None:
                self._table.resize(0)
            slot.update(run_number)
        indices = slot.created.next(step_size) # /!\ 
        steps = indices_len(indices) # /!\
        if steps==0:
            return self._return_run_step(self.state_blocked, steps_run=0)
        op = data.loc[fix_loc(indices)].max(keepdims=False) # /!\
        # ...